<a href="https://colab.research.google.com/github/YoungriKIM/1D-Speech-Emotion-Recognition/blob/master/p_e_detector_model_02_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 화자 추가 학습에 따른 가중치 비교용 모델 작성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 라이브러리
import numpy as np
import os
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras import optimizers
from keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, LSTM, Dense, Activation, Layer
from keras.layers import Conv2D, MaxPooling2D, Reshape, Flatten, LeakyReLU
import random
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.models import load_model
! pip install keras_self_attention
from keras_self_attention import SeqSelfAttention
import datetime


In [ ]:
# 랜덤시드 고정 -------------------------------
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
# --------------------------------------------

In [ ]:
# 데이터 불러와서 합치기

p_x_tr_df = np.load('/content/drive/MyDrive/SER/kesdy18/npy/pitch_x_tr_df.npy')
e_x_tr_df = np.load('/content/drive/MyDrive/SER/kesdy18/npy/energy_x_tr_df.npy')
p_x_t_df = np.load('/content/drive/MyDrive/SER/kesdy18/npy/pitch_x_t_df.npy')
e_x_t_df = np.load('/content/drive/MyDrive/SER/kesdy18/npy/energy_x_t_df.npy')

print(p_x_t_df.shape)
print(e_x_t_df.shape)

x_tr = np.concatenate((p_x_tr_df, e_x_tr_df), axis=1)
y_tr = np.load('/content/drive/MyDrive/SER/kesdy18/npy/pitch/15_kesd_ind_pitch_y_train.npy', allow_pickle=True)
y_tr_n = np.load('/content/drive/MyDrive/SER/kesdy18/npy/pitch/15_kesd_ind_pitch_train_name.npy', allow_pickle=True)

x_t = np.concatenate((p_x_t_df, e_x_t_df), axis=1)
y_t = np.load('/content/drive/MyDrive/SER/kesdy18/npy/pitch/15_kesd_ind_pitch_y_test.npy', allow_pickle=True)
y_t_n = np.load('/content/drive/MyDrive/SER/kesdy18/npy/pitch/15_kesd_ind_pitch_test_name.npy', allow_pickle=True)


x_tr = x_tr.reshape(x_tr.shape[0], x_tr.shape[1], 1)
x_t = x_t.reshape(x_t.shape[0], x_t.shape[1], 1)

print(x_tr.shape, y_tr.shape, y_tr_n.shape)
print(x_t.shape, y_t.shape, y_t_n.shape)

(480, 4)
(480, 5)
(1920, 9, 1) (1920, 4) (1920,)
(480, 9, 1) (480, 4) (480,)


In [ ]:
# https://github.com/YoungriKIM/STUDY/blob/d506036d0d0a19334fd2fe5ab0e6ec52f0c10839/ml/m17_nested_cv.py
# 랜덤시드 고정 -------------------------------
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
# --------------------------------------------
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.utils.testing import all_estimators 
from sklearn.datasets import load_iris 
import warnings

warnings.filterwarnings('ignore')  

X = np.concatenate((x_tr, x_t), axis=0)
Y = np.concatenate((y_tr, y_t), axis=0)
X = X.reshape(X.shape[0], X.shape[1])


kfold = KFold(n_splits=5, shuffle=True)

parameters = [  
    {'n_estimators' : [100,200], 'n_jobs' : [-1]},
    {'max_depth' : [6,8,10,12], 'min_samples_split' : [2,3,5,10]},
    {'min_samples_leaf' : [3,5,7,10]},
    {'min_samples_split' : [2,3,5,10], 'max_depth' : [6,8,10,12]},
    {'n_jobs' : [-1], 'min_samples_leaf' : [3,5,7,10]}
]

#2. 모델 구성
model = GridSearchCV(RandomForestClassifier(), parameters, cv = kfold)

score = cross_val_score(model, X, Y, cv=kfold) 

print('교차검증점수: ', score)

In [ ]:
# 데이터 화자 확인용
def check_act(data):
    check_list=[]
    for i in data:
        if i[:3] not in check_list:
            check_list.append(i[:3])
    print(check_list)

print('x_tr 의 화자: ', end='');check_act(y_tr_n)
print('x_t 의 화자: ', end='');check_act(y_t_n)

x_tr 의 화자: ['004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030']
x_t 의 화자: ['001', '002', '003', '016', '017', '018']


In [ ]:
h5_path = '/content/drive/MyDrive/h5/p_e_detector_model_01.h5'

In [ ]:
def train(model, x_tr, y_tr, epochs, batch_size, h5_path):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=16)
    lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, mode='min')
    mc = ModelCheckpoint(filepath=h5_path, monitor='val_loss', mode='min', verbose=1,
                         save_best_only=True)
    tb = TensorBoard(log_dir='D:/yespeechdata/graph/' + h5_path.split('/')[-1][:-3] + "/",histogram_freq=0, write_graph=True, write_images=True)
    
    history = model.fit(x_tr, y_tr, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[es, mc, tb, lr])

    print('loss : ', history.history['loss'][-1])
    print('acc : ', history.history['accuracy'][-1])
    print('val loss : ', history.history['val_loss'][-1])
    print('val acc : ', history.history['val_accuracy'][-1])
    return model

def test(model, x_t, y_t):
    saved_model = load_model(model, custom_objects={'SeqSelfAttention':SeqSelfAttention})
    print(x_t.shape)
    print(y_t.shape)
    score = saved_model.evaluate(x_t, y_t, batch_size=16)
    print('test loss : ', score[0])
    print('test acc : ', score[1])

    return score

In [ ]:
def cf_make(cf_matrix):
    
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt

    # 시각화
    categories = ['anger','happiness', 'neutral', 'sadness']
    group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
    group_percentages = []
    for i in cf_matrix:
        group_percentage = ["{0:.2%}".format(value) for value in i.flatten()/np.sum(i)]
        group_percentages.append(group_percentage)
    
    group_percentageslist = sum(group_percentages, [])

    # average acc 구하기 -------------------
    sumcm = 0
    cate = cf_matrix.shape[0]
    for j in range(cate):
        for k in range(cate):
            if j == k:
                thenum = group_percentages[j][k]
                sumcm += float(thenum[:-1])
                aveacc = sumcm/cate
    print('average acc: ', round(aveacc, 2),'%')
    # -------------------------------------
    # 시각화 이어서 진행
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_counts,group_percentageslist)]
    labels = np.asarray(labels).reshape(4,4)
    plt.subplots(figsize=(10,6))
    plt.title('confusion_matrix')
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues', xticklabels=categories, yticklabels=categories)
    plt.show()

In [ ]:
'''
# 랜덤시드 고정 -------------------------------
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
# --------------------------------------------

# 시간 측정
str_time = datetime.datetime.now()

num_batch_size = 16
num_epochs = 1500
learning_rate = 0.0001

# define model
model = emo1d(input_shape=x_tr.shape[1:], num_classes=len(np.unique(np.argmax(y_tr, 1))), lr = learning_rate)
# model.summary()

# train model
model = train(model, x_tr, y_tr, num_epochs, num_batch_size, h5_path)

# test model
print('='*100)
score = test(h5_path, x_t, y_t)

# predict model
saved_model = load_model(h5_path, custom_objects={'SeqSelfAttention':SeqSelfAttention})
y_p = saved_model.predict(x_t)
y_p = np.argmax(y_p, axis=1)
y_t = np.argmax(y_t, axis=1)
cf_matrix = confusion_matrix(y_t, y_p)
cf_make(cf_matrix)

# 시간 측정
print('모델 학습이 완료되었습니다.')
print('time: ', datetime.datetime.now() - str_time)
'''

"\n# 랜덤시드 고정 -------------------------------\nSEED = 42\nos.environ['PYTHONHASHSEED'] = str(SEED)\nos.environ['TF_DETERMINISTIC_OPS'] = '1'\ntf.random.set_seed(SEED)\nnp.random.seed(SEED)\nrandom.seed(SEED)\n# --------------------------------------------\n\n# 시간 측정\nstr_time = datetime.datetime.now()\n\nnum_batch_size = 16\nnum_epochs = 1500\nlearning_rate = 0.0001\n\n# define model\nmodel = emo1d(input_shape=x_tr.shape[1:], num_classes=len(np.unique(np.argmax(y_tr, 1))), lr = learning_rate)\n# model.summary()\n\n# train model\nmodel = train(model, x_tr, y_tr, num_epochs, num_batch_size, h5_path)\n\n# test model\nprint('='*100)\nscore = test(h5_path, x_t, y_t)\n\n# predict model\nsaved_model = load_model(h5_path, custom_objects={'SeqSelfAttention':SeqSelfAttention})\ny_p = saved_model.predict(x_t)\ny_p = np.argmax(y_p, axis=1)\ny_t = np.argmax(y_t, axis=1)\ncf_matrix = confusion_matrix(y_t, y_p)\ncf_make(cf_matrix)\n\n# 시간 측정\nprint('모델 학습이 완료되었습니다.')\nprint('time: ', datetime.datetime

In [ ]:
# =================================================================================